# PyTorch Model Performance 🔥

## Housekeeping 🏠

In [40]:
# Install packages as needed.

# %pip install ipywidgets
# %conda install ipykernel
# %conda install pytorch
# %conda install torchvision
# %conda install torchaudio
# %conda install torchserve
# %conda install cudatoolkit=11.3
# %conda install sklearn
# %pip install transformers

In [41]:
# Import packages
import transformers
import torch
import torch.nn as nn

## HuggingFace Example 🤗 

### Standard Approach

In [43]:
%%time
bert = transformers.BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CPU times: user 1.27 s, sys: 297 ms, total: 1.57 s
Wall time: 1.55 s


### Alternate approach

In [44]:
%%time
# Load the model back in.
bert_2 = torch.load("bert.pt")

CPU times: user 16.2 ms, sys: 113 ms, total: 129 ms
Wall time: 128 ms


## Diving Deep 🤿

_Saving and loading a model involves **serialization** during saving and **deserialization** during initialization or loading_

_PyTorch uses the 🐍 Python's **pickle** module to save and load the model. **Pickle** is imported automatically when you_ `import torch`

### Define a Model

In [45]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        
        self.sequential = nn.Sequential(nn.Conv2d(1, 32, 5), 
                                        nn.Conv2d(32, 64, 5), 
                                        nn.Dropout(0.3))
        self.layer1 = nn.Conv2d(64, 128, 5)
        self.layer2 = nn.Conv2d(128, 256, 5)
        self.fc = nn.Linear(256*34*34, 128)
    
    def forward(self, x):
        
        output = self.sequential(x)
        output = self.layer1(output)
        output = self.layer2(output)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        

In [46]:
# Initialize a model
model = NeuralNet()

In [47]:
# Print
print(model)

NeuralNet(
  (sequential): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (2): Dropout(p=0.3, inplace=False)
  )
  (layer1): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (layer2): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
  (fc): Linear(in_features=295936, out_features=128, bias=True)
)


### Saving (Standard) 💾
Saving `state_dict`. This will serialize the model and save it

In [48]:
torch.save(model.state_dict(), 'model1.pt')

#### Inside `state_dict()` 👀

In [49]:
type(model.state_dict())

collections.OrderedDict

In [23]:
model.state_dict().keys()

odict_keys(['sequential.0.weight', 'sequential.0.bias', 'sequential.1.weight', 'sequential.1.bias', 'layer1.weight', 'layer1.bias', 'layer2.weight', 'layer2.bias', 'fc.weight', 'fc.bias'])

### Loading (Standard) 🏋🏾‍♂️

In [50]:
%%time
# Instantiate the model
model_s1 = NeuralNet()
# Load the weights
model.load_state_dict(torch.load('model1.pt'))

CPU times: user 243 ms, sys: 43.1 ms, total: 286 ms
Wall time: 271 ms


<All keys matched successfully>

### Saving (Alternate)  🗜
Save the entire model along with the class using `torch.save`

In [51]:
torch.save(model,'model2.pt')

### Loading (Alternate) 🚀

In [52]:
%%time
model_a2 = torch.load('model2.pt')

CPU times: user 5.14 ms, sys: 36.7 ms, total: 41.9 ms
Wall time: 40.9 ms


In [53]:
# Serialize the model we loaded in the previous code listing.
#torch.save(bert, "bert.pt")

##  🗝 Takeaways

- Use the `state_dict()` method when you want to optimize for probability like federated learning
- Use the `torch.save` method when you optimize for performance like distributed training where portability is not a constraint and resiliency is handled by other mechanisms  
- Do a `model.eval()` after importing the weights. This will set the dropout and the normalization to eval
